In [3026]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
from dash import Dash,dcc, html, callback,Input,Output,callback_context,no_update
import dash_bootstrap_components as dbc
import dash_ag_grid as dag

#### In this small project i want to make analyzing using oop

**Let's start!**

In [3027]:
app = Dash(__name__,external_stylesheets=[dbc.themes.BOOTSTRAP])
app.config.suppress_callback_exceptions = True

In [3028]:
df = pd.read_csv("Sample - Superstore.csv",encoding='cp1252') # <- Just reading file

In [3029]:
dfinfo = pd.read_csv('Info.csv')

In [3030]:
df.isna().sum() # We haven't any null here and we can continue work with this project

Row ID           0
Order ID         0
Order Date       0
Ship Date        0
Ship Mode        0
Customer ID      0
Customer Name    0
Segment          0
Country          0
City             0
State            0
Postal Code      0
Region           0
Product ID       0
Category         0
Sub-Category     0
Product Name     0
Sales            0
Quantity         0
Discount         0
Profit           0
dtype: int64

In [3031]:
df.head(1)

,Row ID,Order ID,Order Date,Ship Date,Ship Mode,Customer ID,Customer Name,Segment,Country,City,...,Postal Code,Region,Product ID,Category,Sub-Category,Product Name,Sales,Quantity,Discount,Profit
0,1,CA-2016-152156,11/8/2016,11/11/2016,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,...,42420,South,FUR-BO-10001798,Furniture,Bookcases,Bush Somerset Collection Bookcase,261.96,2,0.0,41.9136


In [3032]:
df_cat = df.groupby('Category')['Sales'].sum().reset_index()

In [3033]:
df_cat

,Category,Sales
0,Furniture,741999.7953
1,Office Supplies,719047.0320
2,Technology,836154.0330


## Page 0

In [3034]:
df_data = df.copy()

In [3035]:
df_data['Order Date'] = pd.to_datetime(df_data['Order Date'])
df_data = df_data[['Order Date','Sales','Profit']]

In [3036]:
df_data[df_data['Order Date'].between('2014-01-03','2014-01-05')]

,Order Date,Sales,Profit
739,2014-01-04,11.784,4.2717
740,2014-01-04,272.736,-64.7748
741,2014-01-04,3.540,-5.4870
1759,2014-01-05,19.536,4.8840
7980,2014-01-03,16.448,5.5512


In [3037]:
df_data = df_data.groupby('Order Date')[['Sales','Profit']].sum().reset_index()

In [3038]:
df_data['date_num'] = (df_data['Order Date']-pd.to_datetime('2014-01-03')).dt.days

In [3039]:
df_data = df_data.sort_values(by='Order Date')

In [3040]:
df_data.head()

,Order Date,Sales,Profit,date_num
0,2014-01-03,16.448,5.5512,0
1,2014-01-04,288.060,-65.9901,1
2,2014-01-05,19.536,4.8840,2
3,2014-01-06,4407.100,1358.0524,3
4,2014-01-07,87.158,-71.9621,4


In [3041]:
marks = {i : date.strftime(r'%Y-%m-%d') for i,date in zip(df_data['date_num'],df_data['Order Date'])}

In [3042]:
card1 = dbc.Card(
    dbc.CardBody([
        html.H4("Total Sales", className="card-title"),
        html.H2(id='text-card1', className="card-text"),
        html.I(className="bi bi-info-circle-fill me-2"),
    ]),
    id = 'card1',
    color="primary", inverse=True
)

In [3043]:
card2 = dbc.Card(
    dbc.CardBody([
        html.H4("Total Profit", className="card-title"),
        html.H2(id='text-card2', className="card-text"),
        html.I(className="bi bi-info-circle-fill me-2"),
    ]),
    id = 'card2',
    color="primary", inverse=True
)

In [3044]:
range_slider_card1 = dcc.RangeSlider(id = 'range-slider',
                    min=df_data['date_num'].min(),max=df_data['date_num'].max(),step=1,
                    value = [df_data['date_num'].min(),df_data['date_num'].max()],marks=None)

In [3045]:

page_0 = dbc.Container(
    [
    dbc.Row([
    html.H3('Category sales', className='text-left'),
    ]),
dbc.Row([
    dbc.Col(),
    dbc.Col(dcc.RadioItems(
        ['Sales','Profit'],id='sales_or_profit',value='Sales',
        inline=True)),
    ]),
dbc.Row([
    dbc.Col(dcc.Graph(id='categories'),
                  width={"size": 4}),
    dbc.Col(dcc.Graph(id='sub-categories'),width={"size": 4}),
    dbc.Col(dcc.Graph(id='one-sales-profit'),width={"size": 4}),
    ]),
dbc.Row([
    dbc.Col(card1),
    dbc.Col(card2),
    dbc.Col(),
    dbc.Col(),
]),
dbc.Row([
    dbc.Col(range_slider_card1),
]),
dbc.Row([
    dbc.Col(html.Div(id='slider-output', className='mt-4')),
        ]),
   ],fluid=True
   )

## Page 1

In [3046]:
superstore_cities = pd.read_csv("Superstore cities.csv")

In [3047]:
import json
with open('state_codes.json', 'r') as f:
    state_codes = json.load(f)

In [3048]:
df_states_w_city = df.drop('Order Date',axis=1).groupby(['State','City']).sum()[['Sales','Profit']].reset_index()

In [3049]:
df_states_w_city['State code'] = df_states_w_city['State'].map(state_codes)

In [3050]:
df_states_no_city = df_states_w_city.drop('City',axis=1).groupby(['State','State code']).sum().reset_index()

In [3051]:
df_states_no_city.head()

,State,State code,Sales,Profit
0,Alabama,AL,19510.6400,5786.8253
1,Arizona,AZ,35282.0010,-3427.9246
2,Arkansas,AR,11678.1300,4008.6871
3,California,CA,457687.6315,76381.3871
4,Colorado,CO,32108.1180,-6527.8579


In [3052]:
df_states_no_city['Profit'] = df_states_no_city['Profit'].round().apply(lambda x: f'{x} $')

In [3053]:
df_states_no_city['Sales'] = df_states_no_city['Sales'].round()

In [3054]:
mapp = px.choropleth(
    df_states_no_city,
    locations='State code',
    locationmode="USA-states",
    color='Sales',
    scope="usa",
    hover_name='State',
    hover_data={'Sales': True, 'Profit': True},
    color_continuous_scale='Viridis'
)

In [3055]:
page_1 = dbc.Container([
    dbc.Row([
    html.H3('Sales map', className='text-left'),
    ]),
    dbc.Row([
    dbc.Col(dcc.Graph(figure=mapp,id='map',
            style={'padding': '0px', 'margin': '0px','height': '100vh'},
             className='container-fluid'),width=7
    ),
    dbc.Col([
        dbc.Row(
            dcc.Graph(figure = {},id='map_state',
            style={'padding': '0px', 'margin': '0px','height': '45vh'},
            className='container-fluid'),
        style={'height': '50%'},
        ),
        dbc.Row(
        dcc.Graph(figure = {},id='sub-category-in-state',
                 className='container-fluid',
                 style={'padding': '0px', 'margin': '0px','height': '45vh'},),    
        style={'height': '50%'},
        ),
        ]),   
    ]),
],fluid=True, className="mt-4")

In [3056]:
app.layout = dbc.Container([
    dcc.Location(id='url', refresh=False),
    dbc.NavbarSimple(
        brand="Superstore",
        brand_href="/",
        color="primary",
        dark=True,
        children=[
            dbc.NavItem(dcc.Link('Главная', href='/', className='nav-link')),
            dbc.NavItem(dcc.Link('Страница 1', href='/page-1', className='nav-link')),
        ]
    ),
    html.Div(id='page-content')
], fluid=True)

In [3057]:
@app.callback(Output('page-content', 'children'),
              [Input('url', 'pathname')])
def display_page(pathname):
    if pathname == '/':
        return page_0
    elif pathname == '/page-1':
        return page_1

In [3058]:
@callback(
    Output('sub-categories','figure'),
    Output('one-sales-profit','figure'),
    Output('categories','figure'),
    Input('categories','clickData'),
    Input('sales_or_profit','value'),
    Input('range-slider','value'),
)
def update_bar_char(clickData,val,date):
    start_date = pd.to_datetime('2014-01-01') + pd.to_timedelta(date[0], unit='D')
    end_date = pd.to_datetime('2014-01-01') + pd.to_timedelta(date[1], unit='D')
    df['Order Date'] = pd.to_datetime(df['Order Date'])
    df_subs_cat = df[df['Order Date'].between(start_date,end_date)].drop('Order Date',axis=1).\
    groupby(['Sub-Category','Category']).sum()[['Sales','Quantity','Profit']].reset_index()
    if clickData == None:
         selected_category = 'Furniture'
    else :
        selected_category = clickData['points'][0]['label']       
    filtered_df = df_subs_cat[df_subs_cat['Category'] == selected_category]    
    fig = px.bar(filtered_df.sort_values(by=val), y='Sub-Category', x=val, title=f'{val} in {selected_category}')
    filtered_df2 = filtered_df.copy()
    filtered_df2['coef'] = filtered_df2['Profit']/filtered_df2['Sales']
    fig2 = px.bar(filtered_df2.sort_values(by='coef'),x='Sub-Category',y='coef',title='Profit by one sale')
    fig3 = px.pie(data_frame=df_subs_cat,values=val,names='Category',title='Category')
    return fig,fig2,fig3

In [3059]:
@callback(
    Output('text-card1','children'),
    Output('text-card2','children'),
    Input('range-slider','value')
)
def cards(date):
    start_date = pd.to_datetime('2014-01-01') + pd.to_timedelta(date[0], unit='D')
    end_date = pd.to_datetime('2014-01-01') + pd.to_timedelta(date[1], unit='D')
    df['Order Date'] = pd.to_datetime(df['Order Date'])
    df_subs_cat = df[df['Order Date'].between(start_date,end_date)].drop('Order Date',axis=1).\
    groupby(['Sub-Category','Category']).sum()[['Sales','Quantity','Profit']].reset_index()
    sales = df_subs_cat['Sales'].sum().round()
    profit = df_subs_cat['Profit'].sum().round()
    return f'{sales} $',f'{profit} $'


In [3060]:
@callback(
    Output('slider-output','children'),
    Input('range-slider','value'),
)
def range_slider(val):
    start_date = pd.to_datetime('2014-01-01') + pd.to_timedelta(val[0], unit='D')
    end_date = pd.to_datetime('2014-01-01') + pd.to_timedelta(val[1], unit='D')
    return f'Selected range: {start_date.date()} to {end_date.date()}'

In [3061]:
@app.callback(
    Output('map_state', 'figure'),
    Input('map', 'clickData')
)
def update_bar_chart(clickData):
    if clickData is None:
        state_name = 'California'
    else:
        state_name = clickData['points'][0]['hovertext']
    filtered_df = df_states_w_city[df_states_w_city['State'] == state_name].sort_values(by='Sales')
    fig = px.bar(filtered_df, x='City', y='Sales')
    fig.update_yaxes(range=[0, 10000])
    return fig

In [3062]:
@callback(
    Output('sub-category-in-state','figure'),
    Input('map', 'clickData'),
)
def update_bar_subcategory(clickData):
    if clickData is None:
        state_name = 'California'
    else:
        state_name = clickData['points'][0]['hovertext']
    df_sub_category_state = df[df['State'] == state_name].drop('Order Date',axis=1).\
    groupby(by='Sub-Category').sum().reset_index()[['Sub-Category','Sales']].sort_values(by='Sales')
    print(df_sub_category_state)
    fig = px.bar(df_sub_category_state, x='Sub-Category', y='Sales')
    return fig


In [3065]:
df['Product Name'].value_counts()[:10]

Product Name
Staple envelope                                              48
Staples                                                      46
Easy-staple paper                                            46
Avery Non-Stick Binders                                      20
Staples in misc. colors                                      19
Staple remover                                               18
KI Adjustable-Height Table                                   18
Storex Dura Pro Binders                                      17
Staple-based wall hangings                                   16
Logitech 910-002974 M325 Wireless Mouse for Web Scrolling    15
Name: count, dtype: int64

In [3063]:
if __name__ == "__main__":
    app.run_server(debug=True,port=8051)

   Sub-Category       Sales
8     Fasteners    479.5300
10       Labels   2899.1400
7     Envelopes   3237.3400
2           Art   5495.6600
15     Supplies  15720.9500
12        Paper  16757.9500
9   Furnishings  19996.4800
1    Appliances  24175.9200
6       Copiers  24559.5200
4     Bookcases  27558.5215
3       Binders  28473.1680
11     Machines  29492.0160
0   Accessories  37255.0100
14      Storage  45112.2900
16       Tables  45430.2320
5        Chairs  63079.3680
13       Phones  67964.5360
  Sub-Category   Sales
5    Fasteners    6.54
7     Supplies   25.50
6       Labels   34.85
2      Binders  114.51
3    Bookcases  141.96
4       Chairs  182.94
0  Accessories  392.94
1   Appliances  416.32
   Sub-Category       Sales
8     Fasteners    479.5300
10       Labels   2899.1400
7     Envelopes   3237.3400
2           Art   5495.6600
15     Supplies  15720.9500
12        Paper  16757.9500
9   Furnishings  19996.4800
1    Appliances  24175.9200
6       Copiers  24559.5200
4     Boo